In [9]:
import json
from pathlib import Path

def coco_to_yolo_pose11(coco_json_path, output_dir):
    with open(coco_json_path, 'r') as f:
        data = json.load(f)

    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True, parents=True)

    # 建立image_id到size和文件名的映射
    id2img = {img['id']: img for img in data['images']}

    for ann in data['annotations']:
        img_info = id2img[ann['image_id']]
        img_name = img_info['file_name']
        img_w = img_info['width']
        img_h = img_info['height']

        # 检查关键点数
        keypoints = ann['keypoints']
        assert len(keypoints) == 6, f"{img_name}关键点数不是2个，请检查COCO标注！"

        # --- 转换目标框 ---
        x_min, y_min, width, height = ann['bbox']
        x_center = (x_min + width / 2) / img_w
        y_center = (y_min + height / 2) / img_h
        norm_width = width / img_w
        norm_height = height / img_h

        # --- 转换关键点 ---
        yolo_kpts = []
        for i in range(0, len(keypoints), 3):
            x = keypoints[i] / img_w
            y = keypoints[i+1] / img_h
            v = int(keypoints[i+2]) if keypoints[i+2] > 0 else 0
            yolo_kpts.extend([x, y, v])

        # 组合成YOLO格式行
        line = [
            0,  # 假设只有一个类别
            x_center,
            y_center,
            norm_width,
            norm_height,
            *yolo_kpts
        ]

        # 写入标签文件
        label_path = output_dir / f"{Path(img_name).stem}.txt"
        with open(label_path, 'a') as f:  # 有多目标时追加
            f.write(' '.join(map(str, line)) + '\n')

# 使用示例
coco_json_path = "/home/hope/R&D_Pro/HRNet/data/bsf2023/annotations/coco_val_cat0.json"
output_dir = "/home/hope/R&D_Pro/HRNet/data/bsf2023/labels/val"
coco_to_yolo_pose11(coco_json_path, output_dir)
